In [1]:
import pandas as pd
import re
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
from tensorflow.keras.optimizers import Adam

In [2]:
df = pd.read_csv("cb_tweets.csv")

In [3]:
df = df.rename(columns={"tweet_text": "text", "cyberbullying_type": "label"})
df['label'] = df['label'].apply(lambda x: 0 if x == 'not_cyberbullying' else 1)

def clean_text(text):
    text = str(text).lower()
    text = re.sub(r'http\S+', '', text)        # remove URLs
    text = re.sub(r'@\w+', '', text)           # remove mentions
    text = re.sub(r'#\w+', '', text)           # remove hashtags
    text = re.sub(r'[^a-z\s]', '', text)       # remove punctuation/numbers
    text = re.sub(r'\s+', ' ', text).strip()   # remove extra spaces
    return text
df['clean_text'] = df['text'].apply(clean_text)

In [4]:
X_train, X_test, y_train, y_test = train_test_split(
    df['clean_text'], df['label'], test_size=0.2, random_state=42, stratify=df['label']
)

MAX_VOCAB = 10000
MAX_LEN = 50

tokenizer = Tokenizer(num_words=MAX_VOCAB, oov_token="<OOV>")
tokenizer.fit_on_texts(X_train)

In [5]:
X_train_seq = tokenizer.texts_to_sequences(X_train)
X_test_seq = tokenizer.texts_to_sequences(X_test)

X_train_pad = pad_sequences(X_train_seq, maxlen=MAX_LEN, padding='post', truncating='post')
X_test_pad = pad_sequences(X_test_seq, maxlen=MAX_LEN, padding='post', truncating='post')

In [6]:
model = Sequential([
    Embedding(MAX_VOCAB, 128, input_length=MAX_LEN),
    LSTM(128, dropout=0.2, recurrent_dropout=0.2, return_sequences=False),
    Dense(64, activation='relu'),
    Dropout(0.3),
    Dense(1, activation='sigmoid')
])

model.compile(loss='binary_crossentropy', optimizer=Adam(1e-3), metrics=['accuracy'])
model.summary()

C:\Users\sauravsuman01\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\core\embedding.py:97: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding (Embedding)                │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm (LSTM)                          │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ ?                           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ ?                           │     0 (unbuilt) │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

In [7]:
history = model.fit(
    X_train_pad, y_train,
    epochs=5,
    batch_size=64,
    validation_split=0.2,
    verbose=1
)

Epoch 1/5
477/477 ━━━━━━━━━━━━━━━━━━━━ 35s 68ms/step - accuracy: 0.8331 - loss: 0.3562 - val_accuracy: 0.8354 - val_loss: 0.2840
Epoch 2/5
477/477 ━━━━━━━━━━━━━━━━━━━━ 45s 77ms/step - accuracy: 0.8439 - loss: 0.2747 - val_accuracy: 0.8328 - val_loss: 0.2845
Epoch 3/5
477/477 ━━━━━━━━━━━━━━━━━━━━ 41s 78ms/step - accuracy: 0.8700 - loss: 0.2503 - val_accuracy: 0.8465 - val_loss: 0.3117
Epoch 4/5
477/477 ━━━━━━━━━━━━━━━━━━━━ 44s 85ms/step - accuracy: 0.8837 - loss: 0.2258 - val_accuracy: 0.8492 - val_loss: 0.3259
Epoch 5/5
477/477 ━━━━━━━━━━━━━━━━━━━━ 41s 85ms/step - accuracy: 0.9008 - loss: 0.1997 - val_accuracy: 0.8530 - val_loss: 0.3305


In [8]:
y_pred_prob = model.predict(X_test_pad)
y_pred = (y_pred_prob > 0.5).astype("int32")

print("\n📊 Classification Report:")
print(classification_report(y_test, y_pred, target_names=['non-bullying', 'bullying']))

acc = accuracy_score(y_test, y_pred)
print(f"✅ Test Accuracy: {acc:.4f}")

299/299 ━━━━━━━━━━━━━━━━━━━━ 6s 17ms/step

📊 Classification Report:
              precision    recall  f1-score   support

non-bullying       0.53      0.50      0.52      1589
    bullying       0.90      0.91      0.91      7950

    accuracy                           0.84      9539
   macro avg       0.72      0.71      0.71      9539
weighted avg       0.84      0.84      0.84      9539

✅ Test Accuracy: 0.8435


The classification report shows that the LSTM model achieved an overall accuracy of 83% in detecting cyberbullying tweets. It performs strongly for the bullying class, with a precision of 0.90, recall of 0.89, and F1-score of 0.90, indicating high reliability in identifying harmful content. However, performance for the non-bullying class is weaker, with a precision of 0.49 and recall of 0.53, mainly due to class imbalance. The macro average F1-score (0.70) reflects uneven performance across classes, while the weighted average (0.83) matches overall accuracy, dominated by the majority class.